In [1]:
from Network_PER import *
from DriveSimTrace import *

pygame 2.1.2 (SDL 2.0.18, Python 3.10.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#시뮬레이션 학습 코드
EXP_COUNT = 1
n_runs = 10000
rwd_components = 1
train_scores = np.zeros([EXP_COUNT,n_runs//100])
train_losses = np.zeros([EXP_COUNT,n_runs//100])

for exps in range(EXP_COUNT):
    wins = 0
    per = True
    lrate = 0.0001
    print("learning rate:", lrate)
    agent = Agent(lr=lrate, gamma=0.95, n_actions=5, batch_size=32, epsilon=1.0, input_dims=[1,7], per_on=per)



    sim = DriveSimulator()
    epsilons = []
    avg_score = 0.0
    avg_loss = 0.0



    replace = np.array([])
    replace = np.append(replace, np.asarray(range(0,10001,100)))
    #print(replace)

    for i in range(n_runs):
        if i in replace:
            agent.set_next_weight()
            agent.save_model(f'./models/q_20230226({exps})') #####################

        over = False
        sim.reset(frame_rate=6000)

        state = sim.get_sim_state()
        while not over:
            #print(state[0,1])
            action, pred, pred_C = agent.choose_action(state)
            #print(action, pred)
            state_, stpRwd, sim_over = sim.step(action, pred_C)
            over = sim_over
            
            agent.store_transition(state, action, stpRwd, state_, over, pred)
            state = state_

        Loss = agent.learn(exps)
        agent.epsilon = round((1.0 - 0.9 * (float(i)/float(n_runs))),3)

    
        avg_score += np.sum(sim.agtRwd)
        avg_loss += np.sum(Loss)



        if sim.episode_count % 100 == 0:
            avg_score /= 100
            avg_loss /= 100
            print('Episode', sim.episode_count, ', average score(last 100 episodes):', avg_score, \
                'average loss:', avg_loss, 'wins:', sim.win_count, 'eps:', agent.epsilon)

            train_scores[exps, i//100] = avg_score
            train_losses[exps, i//100] = avg_loss
            avg_score = 0.0
            avg_loss = 0.0
            epsilons.append(agent.epsilon)




    agent.save_model(f'./models/q_20230226({exps})') ########################
np.savetxt(f'20230226_scores.csv', train_scores, delimiter=",") ###################
np.savetxt(f'20230226_losses.csv', train_losses, delimiter=",") ###################

learning rate: 0.0001
Use Prioritized Sampling: True
q_next weight set!
saved weights to ./models/q_20230226(0)
Episode 100 , average score(last 100 episodes): -0.85 average loss: 0.03249797673124703 wins: 8 eps: 0.991
q_next weight set!
saved weights to ./models/q_20230226(0)
Episode 200 , average score(last 100 episodes): -0.88 average loss: 0.013153569483911269 wins: 14 eps: 0.982
q_next weight set!
saved weights to ./models/q_20230226(0)
Episode 300 , average score(last 100 episodes): -0.86 average loss: 0.010952980514412047 wins: 21 eps: 0.973
q_next weight set!
saved weights to ./models/q_20230226(0)
Episode 400 , average score(last 100 episodes): -0.66 average loss: 0.00897676895467157 wins: 38 eps: 0.964
q_next weight set!
saved weights to ./models/q_20230226(0)
Episode 500 , average score(last 100 episodes): -0.86 average loss: 0.008797820361469349 wins: 45 eps: 0.955
q_next weight set!
saved weights to ./models/q_20230226(0)
Episode 600 , average score(last 100 episodes): -0.

In [ ]:
#시뮬레이션 학습 코드 (추가 학습)
EXP_COUNT = 1
n_runs = 10000
rwd_components = 4
train_scores = np.zeros([EXP_COUNT,n_runs//100])
train_losses = np.zeros([EXP_COUNT,n_runs//100])

for exps in range(EXP_COUNT):
    wins = 0
    per = True
    lrate = 0.0005
    print("learning rate:", lrate)
    agent = Agent(lr=lrate, gamma=0.999, n_actions=5, batch_size=64, epsilon=1.0, input_dims=[4,5], per_on=per)
    
    model = f'./models/q_20230214(0)'
    agent.load_model(model)

    sim = DriveSimulator()
    epsilons = []
    avg_score = 0.0
    avg_loss = 0.0
    
    for i in range(n_runs):
        over = False
        sim.reset(frame_rate=3000)

        state = sim.get_sim_state()
        while not over:
            action, pred, pred_C = agent.choose_action(state)
            #print(action, pred)
            state_, stpRwd, sim_over = sim.step(action, pred_C)
            over = sim_over
            
            agent.store_transition(state, action, stpRwd, state_, over, pred)
            state = state_
        
        Loss = agent.learn(exps)
        #overestimation detection


        agent.epsilon = round((1.0 - (float(i)/float(n_runs))),3)
        #scores.append(sim.agtRwd)
    
        avg_score += np.sum(sim.agtRwd)
        avg_loss += np.sum(Loss)
        if sim.episode_count % 100 == 0:
            avg_score /= 100
            avg_loss /= 100
            print('Episode', sim.episode_count, ', average score(last 100 episodes):', avg_score, \
                'average loss:', avg_loss, 'wins:', sim.win_count, 'eps:', agent.epsilon)

            train_scores[exps, i//100] = avg_score
            train_losses[exps, i//100] = avg_loss
            avg_score = 0.0
            avg_loss = 0.0
            epsilons.append(agent.epsilon)

    agent.save_model(f'./models/q_20230216({exps})')
np.savetxt(f'20220216_scores.csv', train_scores, delimiter=",")
np.savetxt(f'20220216_losses.csv', train_losses, delimiter=",")

In [ ]:

def save_graph(date, label):
    scores = np.genfromtxt(date + '_scores.csv', delimiter=',')
    losses = np.genfromtxt(date + '_losses.csv', delimiter=',')

    # Plot Score
    plt.title(label + ' (score)')
    for i in range(EXP_COUNT):
        plt.plot(scores[i], 'b')
        plt.plot(8.0 * np.ones_like(scores[i]), 'gray') #Score 상한선
        plt.plot(-5.0 * np.ones_like(scores[i]), 'gray') #Score 하한선
        plt.show()
    #plt.savefig(date + '_score.png')
    #plt.show()

    for i in range(EXP_COUNT):
        plt.title(label + ' (loss)')
        #Plot Loss
        plt.plot(losses[i], 'orange')
        plt.plot(1.5 * np.ones_like(losses[i]), 'gray') #Loss 상한선
        plt.plot(np.zeros_like(losses[i]), 'gray') #Loss 하한선
        plt.show()
    #plt.savefig(date + '_loss.png')
    #plt.show()

save_graph(date = '20221209', label = 'test1')